# 3. Building Compositional Space Complexes to Express Elaborate Design Problems

> ***(before running this code, please consult Quick Start to make sure everything is set up)***

> You will need the same dependencies as in [Tutorial No.2](02.AdditiveManufacturingPathPlanning.ipynb) (i.e., `pycalphad` and `pathfinding`), and additionally `igraph` in order to efficiently visualize the examples presented in this tutorial. **If you are running this in `nimplex`'s Codespaces, everything has been pre-installed for you.**

**In this tutorial, we will build upon `nimplex`'s graph representations and leverage our ability to combine ("stitch") them together without affecting the way we express the problem at hand and in an effortless fashion to dramatically speed up materials exploration.** In particular, we will show how to:

1. **Plot graphs** (as in mathematical objects, not any figures) using [`igraph`](https://igraph.org/), which can be done quite directly using `nimplex`'s point grid and neighbor lists `edges = [(i,n) for i in range(len(gridAtt)) for n in nList[i]]`

2. **Identify ordered subsystems in `nimplex` grids, like `A-B-C` and `C-A-B` within `A-B-C-D-E` and `D-C-G-F-A-H-B` to establish connectivity between them.** We will use this to combine 3 4-component systems (tetrahedra) by 2 3-component subsystems (triangles) to create a chain.

3. Identify all subspaces of a given order in low dimensional and high dimensional spaces (e.g., all quantized compositions of any 3 things out of N) and combine ("stitch") them together to form a **simplex graphs that intersect themselves in 3D because of high dimensionality, yet still have the same (graph) structure**.

4. **Construct a graph complex to explore all ternary combinations of `["Ti50Zr50", "Hf95Ti5", "NbTaWHf", "Mo80Nb10W10", "TiTa2", "Nb96Mo3W1", "Zr49 Hf1 Mo50"]` (7 alloys)** under equilibrium phase constraint (relatively expensive to compute) and then explore the space with an additional low-cost screenig constraint (RMSAD - alloy strenght surrogate) that could also be an ML surrogate.

In [69]:
import nimplex
from utils import stitching
from IPython.display import clear_output
from itertools import combinations

In [70]:
import igraph as ig
import plotly.graph_objs as go
import numpy as np
import random
random.seed(123)

## Plotting 4-Component Simplex Graph with `igraph`

In [71]:
dim = 4
ndiv = 6
gridAtt, nList = nimplex.simplex_graph_py(dim, ndiv)

In [72]:
edges = []
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i,n))

In [73]:
def plotGraph(edges: list):
    # Lets generate some seed positions for the nodes
    G = ig.Graph(edges)
    layout = G.layout_kamada_kawai(dim=3)
    layout_array = np.array(layout.coords)
    # Create edge traces
    edge_traces = []
    for edge in G.es:
        start, end = edge.tuple
        x0, y0, z0 = layout_array[start]
        x1, y1, z1 = layout_array[end]
        width = 5
        edge_trace = go.Scatter3d(x=[x0, x1], y=[y0, y1], z=[z0, z1],
                                line=dict(width=width, color='lightgray'),
                                opacity=0.2,
                                hoverinfo='none', mode='lines')
        edge_traces.append(edge_trace)

    node_trace = go.Scatter3d(x=layout_array[:, 0], y=layout_array[:, 1], z=layout_array[:, 2],
                            mode='markers',
                            marker=dict(size=3, 
                                        opacity=0.5,
                                        line=None,
                                        color='blue'),
                            
                            text=[f"Node {i}" for i in range(len(gridAtt))],
                            hoverinfo='text')

    # Create figure
    fig = go.Figure(data=edge_traces + [node_trace])

    # Update layout
    fig.update_layout(
        title='3D Graph with Feasibility Types',
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        showlegend=False,
        margin=dict(l=0, r=0, b=0, t=0),
        width=800
    )

    fig.show()

In [74]:
#plotGraph(edges)
clear_output()

## Combining ("Stitching") 3 4-Component Systems into A Complex

In [75]:
sys1 = ["A", "B", "C", "D"]
sys2 = ["B", "C", "D", "E"]
sys3 = ["B", "D", "E", "F"]

In [76]:
offset = len(gridAtt)
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i+offset,n+offset))
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i+offset*2,n+offset*2))

In [77]:
#plotGraph(edges)
clear_output()

In [78]:
stitchingPoints = stitching.findStitchingPoints_py(dim, ndiv, components=["A", "B", "C", "D"])
print(stitchingPoints["A-B"])

[83, 82, 79, 73, 63, 48, 27]


In [79]:
stitchingPoints1 = stitching.findStitchingPoints_py(dim, ndiv, components=sys1, offset=0)
stitchingPoints2 = stitching.findStitchingPoints_py(dim, ndiv, components=sys2, offset=len(gridAtt))
stitchingPoints3 = stitching.findStitchingPoints_py(dim, ndiv, components=sys3, offset=len(gridAtt)*2)

In [80]:
for i, j in zip(stitchingPoints1["B-C-D"], stitchingPoints2["B-C-D"]):
    edges.append((i, j))

In [81]:
for i, j in zip(stitchingPoints2["B-D-E"], stitchingPoints3["B-D-E"]):
    edges.append((i, j))

In [82]:
#plotGraph(edges)
clear_output()

## Combining ("Stitching") 9 3-Component Systems into A Complex
### Disregard this section for now Adam

In [83]:
dim = 3
ndiv = 6
nsys = 9
gridAtt, nList = nimplex.simplex_graph_py(dim, ndiv)
edges = []
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i,n))

In [84]:
len(gridAtt)

28

In [85]:
offset = len(gridAtt)
for sys in range(1,nsys+1):
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset*sys,n+offset*sys))

In [86]:
#plotGraph(edges)
clear_output()

In [87]:
from pycalphad import Database
dbCrNiV = Database("ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB")
dbCrFeTi = Database("ammap/databases/Cr-Fe-Ti_wang2017.tdb")
dbCrFeNi = Database("ammap/databases/Cr-Fe-Ni_miettinen1999.tdb")
dbCrNiTi= Database("ammap/databases/Cr-Ni-Ti_huang2018.tdb")
dbCrTiV= Database("ammap/databases/Cr-Ti-V_ghosh2002.tdb")
dbFeNiTi= Database("ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb")
dbFeNiV= Database("ammap/databases/Fe-Ni-V_zhao2014.tdb")
dbFeTiV= Database("ammap/databases/Fe-Ti-V_guo2012.TDB")
dbNiTiV= Database("ammap/databases/Ni-Ti-V_zou2018.tdb")
CrNiV = ['Cr', 'Ni', 'V']
CrFeTi = ['Cr', 'Fe', 'Ti']
CrFeNi = ['Cr', 'Fe', 'Ni']
CrNiTi = ['Cr', 'Ni', 'Ti']
CrTiV = ['Cr', 'Ti', 'V']
FeNiTi = ['Fe', 'Ni', 'Ti']
FeNiV = ['Fe', 'Ni', 'V']
FeTiV = ['Fe', 'Ti', 'V']
NiTiV = ['Ni', 'Ti', 'V']


/opt/conda/lib/python3.11/site-packages/pycalphad/io/tdb.py:293: UserWarning:

The type definition character `&` in `TYPE_DEFINITION & GES A_P_D A2_BCC MAGNETIC -1.0 4.00000E-01 ` is not used by any phase.



In [88]:
stitchingPointsCrNiV = stitching.findStitchingPoints_py(dim, ndiv, components=CrNiV, offset=0)
stitchingPointsCrFeTi = stitching.findStitchingPoints_py(dim, ndiv, components=CrFeTi, offset=len(gridAtt))
stitchingPointsCrFeNi = stitching.findStitchingPoints_py(dim, ndiv, components=CrFeNi, offset=len(gridAtt)*2)
stitchingPointsCrNiTi = stitching.findStitchingPoints_py(dim, ndiv, components=CrNiTi, offset=len(gridAtt)*3)
stitchingPointsCrTiV = stitching.findStitchingPoints_py(dim, ndiv, components=CrTiV, offset=len(gridAtt)*4)
stitchingPointsFeNiTi = stitching.findStitchingPoints_py(dim, ndiv, components=FeNiTi, offset=len(gridAtt)*5)
stitchingPointsFeNiV = stitching.findStitchingPoints_py(dim, ndiv, components=FeNiV, offset=len(gridAtt)*6)
stitchingPointsFeTiV = stitching.findStitchingPoints_py(dim, ndiv, components=FeTiV, offset=len(gridAtt)*7)
stitchingPointsNiTiV = stitching.findStitchingPoints_py(dim, ndiv, components=NiTiV, offset=len(gridAtt)*8)

## Finding and Combining 3-Component Subspaces in 4-Component Space

In [89]:
from itertools import combinations
elements = ["A", "B", "C", "D"]
ternaries = list(combinations(elements, 3))
print(ternaries)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D')]


In [90]:
gridAtt, nList = nimplex.simplex_graph_py(3,12)

In [91]:
edges = []
# Ternaries
for i, ternary in enumerate(ternaries):
    offset = i*len(gridAtt)
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset,n+offset))

In [92]:
plotGraph(edges)
clear_output()

In [93]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('A', 'B'): occurs between ternary [(0, 1)]
('A', 'C'): occurs between ternary [(0, 2)]
('B', 'C'): occurs between ternary [(0, 3)]
('A', 'D'): occurs between ternary [(1, 2)]
('B', 'D'): occurs between ternary [(1, 3)]
('C', 'D'): occurs between ternary [(2, 3)]


In [94]:
for stitchingBinary, ternaryPair in stitchingBinaries.items():
    ternary1, ternary2 = ternaryPair[0][0], ternaryPair[0][1]
    stitching1 = stitching.findStitchingPoints_py(
        3, 12, 
        components=ternaries[ternary1],
        offset=ternary1*len(gridAtt)
        )["-".join(stitchingBinary)]
    stitching2 = stitching.findStitchingPoints_py(
        3, 12, 
        components=ternaries[ternary2],
        offset=ternary2*len(gridAtt)
        )["-".join(stitchingBinary)]
    print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
    for i, j in zip(stitching1, stitching2):
        edges.append((i, j))

Stitching 0 and 1 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103]
Stitching 0 and 2 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 0 and 3 at ('B', 'C') from [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0] to [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285]
Stitching 1 and 2 at ('A', 'D') from [181, 179, 176, 172, 167, 161, 154, 146, 137, 127, 116, 104, 91] to [272, 270, 267, 263, 258, 252, 245, 237, 228, 218, 207, 195, 182]
Stitching 1 and 3 at ('B', 'D') from [103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91] to [363, 361, 358, 354, 349, 343, 336, 328, 319, 309, 298, 286, 273]
Stitching 2 and 3 at ('C', 'D') from [194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182] to [285, 284, 283, 282, 281, 280, 279, 278, 277, 276, 275, 274, 273]


In [95]:
#plotGraph(edges)
clear_output()

## In Higher Dimensional Cases (Stretched 2D Spaces Crossing Each Other in 3D)

In [96]:
from itertools import combinations
elements = ["A", "B", "C", "D", "E"]
ternaries = list(combinations(elements, 3))
print(ternaries)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'B', 'E'), ('A', 'C', 'D'), ('A', 'C', 'E'), ('A', 'D', 'E'), ('B', 'C', 'D'), ('B', 'C', 'E'), ('B', 'D', 'E'), ('C', 'D', 'E')]


In [97]:
gridAtt, nList = nimplex.simplex_graph_py(3,12)

In [98]:
edges = []
# Ternaries
for i, ternary in enumerate(ternaries):
    offset = i*len(gridAtt)
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset,n+offset))


In [99]:
#plotGraph(edges)
clear_output()

In [100]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('A', 'B'): occurs between ternary [(0, 1), (0, 2), (1, 2)]
('A', 'C'): occurs between ternary [(0, 3), (0, 4), (3, 4)]
('B', 'C'): occurs between ternary [(0, 6), (0, 7), (6, 7)]
('A', 'D'): occurs between ternary [(1, 3), (1, 5), (3, 5)]
('B', 'D'): occurs between ternary [(1, 6), (1, 8), (6, 8)]
('A', 'E'): occurs between ternary [(2, 4), (2, 5), (4, 5)]
('B', 'E'): occurs between ternary [(2, 7), (2, 8), (7, 8)]
('C', 'D'): occurs between ternary [(3, 6), (3, 9), (6, 9)]
('C', 'E'): occurs between ternary [(4, 7), (4, 9), (7, 9)]
('D', 'E'): occurs between ternary [(5, 8), (5, 9), (8, 9)]


In [101]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, 12, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, 12, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))

Stitching 0 and 1 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103]
Stitching 0 and 2 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 1 and 2 at ('A', 'B') from [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 0 and 3 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285]
Stitching 0 and 4 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [454, 453, 451, 448, 444, 439, 433, 426, 418, 409, 399, 388, 376]
Stitching 3 and 4 at ('A', 'C') from [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285] to [454, 453, 451, 448, 444, 439, 433, 426, 418, 409, 399, 388, 376]
Stitching 0 and 6 at (

In [102]:
#plotGraph(edges)
clear_output()

## Exploring Complex Formed by All 3-Component Alloy Subsystems of 5-Component Space (w. Infeasibility Gliding) #THIS ONE ADAM

In [103]:
elementalSpaceComponents = ["Cr","Fe","Ni","Ti","V"]
#attainableSpaceComponents = ["SS304L", "Ni", "Cr", "V", "Ti"]
attainableSpaceComponents = ["Fe", "Ni", "Cr", "V", "Ti"]
attainableSpaceComponentPositions = [
    #"Cr","Fe","Ni","Ti","V"
    #[0.1993865031, 0.7044989775,0.096114519430,0,0],
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0,1,0,0,0],
    [0,0,0,0,1],
    [0,0,0,1,0]
]    
ternaries = list(combinations(attainableSpaceComponents, 3))
ternaries_CompPos = list(combinations(attainableSpaceComponentPositions, 3))
ndiv = 8
gridAtt, nList = nimplex.simplex_graph_py(3, ndiv)

for tern, terncp in zip(ternaries, ternaries_CompPos):
    print(f"{str(tern):<40} -> {terncp}")

('Fe', 'Ni', 'Cr')                       -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 1, 0, 0, 0])
('Fe', 'Ni', 'V')                        -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
('Fe', 'Ni', 'Ti')                       -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
('Fe', 'Cr', 'V')                        -> ([0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1])
('Fe', 'Cr', 'Ti')                       -> ([0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0])
('Fe', 'V', 'Ti')                        -> ([0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0])
('Ni', 'Cr', 'V')                        -> ([0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1])
('Ni', 'Cr', 'Ti')                       -> ([0, 0, 1, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0])
('Ni', 'V', 'Ti')                        -> ([0, 0, 1, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0])
('Cr', 'V', 'Ti')                        -> ([0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0])


In [104]:
#From Adam
# Edges list for graph plotting and path finding purposes
edges = []
# Connectivity list within each subsystem
graphN = [[] for i in range(len(gridAtt * len(ternaries)))]
# Connectivity list between subsystems
graphNS = [[] for i in range(len(graphN))]
compositions = []
# Initialize spaces counter
spaces = 0
# Iterate over ternaries
for i, terncp in enumerate(ternaries_CompPos):
    offset = i*len(gridAtt)
    for j in range(len(gridAtt)):
        for n in nList[j]:
            edges.append((j+offset,n+offset))
            graphN[j+offset].append(n+offset)
    gridAttTemp, gridElTemp = nimplex.embeddedpair_simplex_grid_fractional_py(terncp, ndiv)
    compositions += gridElTemp
    spaces += i

In [105]:
spaces

45

In [106]:
# # Edges list for graph plotting and path finding purposes
# edges = []
# # Connectivity list within each subsystem
# graphN = [[] for i in range(len(gridAtt * len(ternaries)))]
# # Connectivity list between subsystems
# graphNS = [[] for i in range(len(graphN))]
# compositions = []
# # Iterate over ternaries
# for i, terncp in enumerate(ternaries_CompPos):
#     offset = i*len(gridAtt)
#     for j in range(len(gridAtt)):
#         for n in nList[j]:
#             edges.append((j+offset,n+offset))
#             graphN[j+offset].append(n+offset)
#     gridAttTemp, gridElTemp = nimplex.embeddedpair_simplex_grid_fractional_py(terncp, ndiv)
#     compositions += gridElTemp

In [107]:
def find_duplicates(lst):
    duplicates_counts = {}
    for item in lst:
        # Use the string representation for complex/unhashable types
        item_str = str(item)
        if item_str in duplicates_counts:
            duplicates_counts[item_str] += 1
        else:
            duplicates_counts[item_str] = 1

    # Filter out items that only appear once
    duplicates = {item: count for item, count in duplicates_counts.items() if count > 1}
    
    if not duplicates:
        return "No duplicates found in the list."
    else:
        result = "Duplicates found:"
        for item, count in duplicates.items():
            result += f"{item} appears {count} times"
        return result.strip()

In [108]:
find_duplicates(compositions)

'Duplicates found:[0.0, 1.0, 0.0, 0.0, 0.0] appears 33 times[0.0, 0.875, 0.125, 0.0, 0.0] appears 12 times[0.0, 0.75, 0.25, 0.0, 0.0] appears 11 times[0.0, 0.625, 0.375, 0.0, 0.0] appears 10 times[0.0, 0.5, 0.5, 0.0, 0.0] appears 9 times[0.0, 0.375, 0.625, 0.0, 0.0] appears 8 times[0.0, 0.25, 0.75, 0.0, 0.0] appears 7 times[0.0, 0.125, 0.875, 0.0, 0.0] appears 6 times[0.0, 0.0, 1.0, 0.0, 0.0] appears 6 times[0.0, 0.0, 0.0, 0.0, 1.0] appears 6 times[0.0, 0.0, 0.125, 0.0, 0.875] appears 3 times[0.0, 0.0, 0.25, 0.0, 0.75] appears 3 times[0.0, 0.0, 0.375, 0.0, 0.625] appears 3 times[0.0, 0.0, 0.5, 0.0, 0.5] appears 3 times[0.0, 0.0, 0.625, 0.0, 0.375] appears 3 times[0.0, 0.0, 0.75, 0.0, 0.25] appears 3 times[0.0, 0.0, 0.875, 0.0, 0.125] appears 3 times[0.0, 0.125, 0.0, 0.0, 0.875] appears 6 times[0.0, 0.125, 0.125, 0.0, 0.75] appears 2 times[0.0, 0.125, 0.25, 0.0, 0.625] appears 2 times[0.0, 0.125, 0.375, 0.0, 0.5] appears 2 times[0.0, 0.125, 0.5, 0.0, 0.375] appears 2 times[0.0, 0.125, 0

In [43]:
print(compositions[0])
print(compositions[17])

[0.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0]


In [44]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('Fe', 'Ni'): occurs between ternary [(0, 1), (0, 2), (1, 2)]
('Cr', 'Fe'): occurs between ternary [(0, 3), (0, 4), (3, 4)]
('Cr', 'Ni'): occurs between ternary [(0, 6), (0, 7), (6, 7)]
('Fe', 'V'): occurs between ternary [(1, 3), (1, 5), (3, 5)]
('Ni', 'V'): occurs between ternary [(1, 6), (1, 8), (6, 8)]
('Fe', 'Ti'): occurs between ternary [(2, 4), (2, 5), (4, 5)]
('Ni', 'Ti'): occurs between ternary [(2, 7), (2, 8), (7, 8)]
('Cr', 'V'): occurs between ternary [(3, 6), (3, 9), (6, 9)]
('Cr', 'Ti'): occurs between ternary [(4, 7), (4, 9), (7, 9)]
('Ti', 'V'): occurs between ternary [(5, 8), (5, 9), (8, 9)]


In [45]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))
            graphNS[i].append(j)

Stitching 0 and 1 at ('Fe', 'Ni') from [44, 43, 41, 38, 34, 29, 23, 16, 8] to [89, 88, 86, 83, 79, 74, 68, 61, 53]
Stitching 0 and 2 at ('Fe', 'Ni') from [44, 43, 41, 38, 34, 29, 23, 16, 8] to [134, 133, 131, 128, 124, 119, 113, 106, 98]
Stitching 1 and 2 at ('Fe', 'Ni') from [89, 88, 86, 83, 79, 74, 68, 61, 53] to [134, 133, 131, 128, 124, 119, 113, 106, 98]
Stitching 0 and 3 at ('Cr', 'Fe') from [0, 9, 17, 24, 30, 35, 39, 42, 44] to [143, 151, 158, 164, 169, 173, 176, 178, 179]
Stitching 0 and 4 at ('Cr', 'Fe') from [0, 9, 17, 24, 30, 35, 39, 42, 44] to [188, 196, 203, 209, 214, 218, 221, 223, 224]
Stitching 3 and 4 at ('Cr', 'Fe') from [143, 151, 158, 164, 169, 173, 176, 178, 179] to [188, 196, 203, 209, 214, 218, 221, 223, 224]
Stitching 0 and 6 at ('Cr', 'Ni') from [0, 1, 2, 3, 4, 5, 6, 7, 8] to [278, 286, 293, 299, 304, 308, 311, 313, 314]
Stitching 0 and 7 at ('Cr', 'Ni') from [0, 1, 2, 3, 4, 5, 6, 7, 8] to [323, 331, 338, 344, 349, 353, 356, 358, 359]
Stitching 6 and 7 at ('Cr'

In [46]:
from pycalphad import Database

# List of database file paths
db_files = [
    "ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB",
    "ammap/databases/Cr-Fe-Ti_wang2017.tdb",
    "ammap/databases/Cr-Fe-Ni_miettinen1999.tdb",
    "ammap/databases/Cr-Ni-Ti_huang2018.tdb",
    "ammap/databases/Cr-Ti-V_ghosh2002.tdb",
    "ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb",
    "ammap/databases/Fe-Ni-V_zhao2014.tdb",
    "ammap/databases/Fe-Ti-V_guo2012.TDB",
    "ammap/databases/Ni-Ti-V_zou2018.tdb"
]

# Dictionary to store unique phases for each database
unique_phases = {}

# Iterate through each database file
for db_file in db_files:
    dbf = Database(db_file)
    phases = list(set(dbf.phases.keys()))
    unique_phases[db_file] = phases

# Print unique phases for each database
for db_file, phases in unique_phases.items():
    print(f"Unique phases for {db_file}: {phases}")

/opt/conda/lib/python3.11/site-packages/pycalphad/io/tdb.py:293: UserWarning:

The type definition character `&` in `TYPE_DEFINITION & GES A_P_D A2_BCC MAGNETIC -1.0 4.00000E-01 ` is not used by any phase.



Unique phases for ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB: ['HIGH_SIGMA', 'M3V', 'BCC_A2', 'HCP_A3', 'B2_BCC', 'L12_FCC', 'LIQUID', 'NI2V', 'NI2V7', 'SIGMA', 'COV3', 'FCC_A1']
Unique phases for ammap/databases/Cr-Fe-Ti_wang2017.tdb: ['BCC_A2', 'BCC_B2', 'HCP_A3', 'C14', 'C15', 'LIQUID', 'SIGMA', 'C36', 'TI5CR7FE17', 'FCC_A1']
Unique phases for ammap/databases/Cr-Fe-Ni_miettinen1999.tdb: ['BCC_A2', 'HCP_A3', 'LIQUID', 'SIGMA', 'FCC_A1']
Unique phases for ammap/databases/Cr-Ni-Ti_huang2018.tdb: ['BCC_A2', 'LAVES_C14', 'LAVES_C15', 'HCP_A3', 'NI3TI', 'LIQUID', 'LAVES_C36', 'NITI', 'NITI2', 'FCC_A1']
Unique phases for ammap/databases/Cr-Ti-V_ghosh2002.tdb: ['BCC_A2', 'LAVES_C14', 'LAVES_C15', 'HCP_A3', 'LIQUID', 'LAVES_C36']
Unique phases for ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb: ['NITI2', 'A2', 'A3', 'NI3TI', 'C14', 'LIQUID', 'FCC4', 'BCC2', 'A1']
Unique phases for ammap/databases/Fe-Ni-V_zhao2014.tdb: ['BCC_A2', 'LAVES_C14', 'A2_BCC', 'A15_NI2V7', 'CBCC_A12', 'FE4N', 'HCP_A3', 'L

In [64]:

dbf = Database("CrHfMoNbTaTiVWZr_9element_Feb2023.tdb")
phases = list(set(dbf.phases.keys()))
print(elementalSpaceComponents)
print(f'Loaded TDB file with phases considered: {phases}')
from myPycalphadCallable import equilibrium_callable

['Ti', 'Zr', 'Hf', 'W', 'Nb', 'Ta', 'Mo']
Loaded TDB file with phases considered: ['BCC_A2', 'LAVES_C14', 'LAVES_C15', 'HCP_A3', 'LIQUID', 'LAVES_C36', 'FCC_A1']


In [65]:
import os
import importlib

# Directory containing the equilibrium files
directory = "ammap/callables/multi_system_equilibrium_and_scheil"

# Get all files starting with "equilibrium"
equilibrium_files = [f for f in os.listdir(directory) if f.startswith("equilibrium") and f.endswith(".py")]

# Dictionary to store imported callables with unique names
equilibrium_callables = {}

# Import each equilibrium file and store the callable with a unique name
for file in equilibrium_files:
    module_name = file[:-3]  # Remove the .py extension
    module_path = f"ammap.callables.multi_system_equilibrium_and_scheil.{module_name}"
    module = importlib.import_module(module_path)
    callable_name = f"{module_name}"
    equilibrium_callables[callable_name] = getattr(module, "equilibrium_callable")

# Print the imported callables
for name, func in equilibrium_callables.items():
    print(f"Imported {name}: {func}")

/opt/conda/lib/python3.11/site-packages/pycalphad/io/tdb.py:293: UserWarning:

The type definition character `&` in `TYPE_DEFINITION & GES A_P_D A2_BCC MAGNETIC -1.0 4.00000E-01 ` is not used by any phase.

/opt/conda/lib/python3.11/site-packages/pycalphad/model.py:1278: UserWarning:

The order-disorder model for "FCC4" has a contribution from the physical property model `magnetic_energy`. Partitioned physical properties are not correctly substituted into the disordered part of the energy. THE GIBBS ENERGY CALCULATED FOR THIS PHASE MAY BE INCORRECT. Please see the discussion in https://github.com/pycalphad/pycalphad/pull/311 for more details.



Imported equilibrium_callable_CrFeTi_08bbfb9a: <function equilibrium_callable at 0x7fea385ab560>
Imported equilibrium_callable_CrTiV_ed4c332b: <function equilibrium_callable at 0x7fea385a8e00>
Imported equilibrium_callable_FeTiV_fa95b3ee: <function equilibrium_callable at 0x7fea23d76200>
Imported equilibrium_callable_FeNiV_b9b0384d: <function equilibrium_callable at 0x7fea23d74400>
Imported equilibrium_callable_CrNiTi_e320ddd5: <function equilibrium_callable at 0x7fea238f23e0>
Imported equilibrium_callable_FeNiTi_06a49695: <function equilibrium_callable at 0x7fea385a85e0>
Imported equilibrium_callable_NiTiV_1d83c99c: <function equilibrium_callable at 0x7fea236a0ea0>
Imported equilibrium_callable_NiCrV_0f8b67c1: <function equilibrium_callable at 0x7fea23078860>
Imported equilibrium_callable_NiCrFe_ce8329fb: <function equilibrium_callable at 0x7fea22a5ee80>


In [66]:
# Define the elemental space components
elementalSpaceComponents = ["Cr", "Fe", "Ni", "Ti", "V"]

# Create a mapping of index positions to their corresponding elemental names
index_to_element = {i: element for i, element in enumerate(elementalSpaceComponents)}

# Assuming 'compositions' is your list of 450 points

# Get the indices of the used dimensions (dimensions where values are non-zero)
used_indices = [np.nonzero(point)[0][:3] for point in compositions]

# Map the used indices to their corresponding elements for each point
used_elements = [[index_to_element[i] for i in indices] for indices in used_indices]

# Display the first 5 results as a sample
print(used_elements)

KeyError: 6

In [ ]:
# Define the elemental space components
elementalSpaceComponents = ["Cr", "Fe", "Ni", "Ti", "V"]

# Create a mapping of index positions to their corresponding elemental names
index_to_element = {i: element for i, element in enumerate(elementalSpaceComponents)}

# Function to find the next ternary space
def find_next_ternary_space(compositions, start_index):
    for i in range(start_index, len(compositions)):
        nonzero_indices = np.nonzero(compositions[i])[0]
        if len(nonzero_indices) >= 3:
            return nonzero_indices[:3]
    return None  # Return None if no suitable ternary space is found

# Process the compositions
used_elements = []
current_ternary_space = None
for i, point in enumerate(compositions):
    nonzero_indices = np.nonzero(point)[0]
    
    if len(nonzero_indices) < 3 and current_ternary_space is None:
        # Find the next ternary space
        current_ternary_space = find_next_ternary_space(compositions, i)
        if current_ternary_space is None:
            # If no suitable ternary space is found, use the first three elements
            current_ternary_space = np.array([0, 1, 2])
    
    if len(nonzero_indices) >= 3:
        # If this point has 3 or more non-zero elements, use them and reset current_ternary_space
        elements = [index_to_element[j] for j in nonzero_indices[:3]]
        current_ternary_space = None
    else:
        # Use the current ternary space
        elements = [index_to_element[j] for j in current_ternary_space]
    
    used_elements.append(elements)

# Display the first 5 results as a sample
print(used_elements)

[['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni', 'V'], ['Fe', 'Ni'

In [ ]:
def implement_representation_logic(compositions, elementalSpaceComponents):
    dimension_map = {i: elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents))}
    used_elements = []
    current_ternary_space = None

    for i, point in enumerate(compositions):
        nonzero_indices = np.nonzero(point)[0]
        len_nonzero = len(nonzero_indices)

        if len_nonzero < 3:
            if current_ternary_space is None or not all(idx in current_ternary_space for idx in nonzero_indices):
                # Find a new ternary space that includes all nonzero indices
                remaining_indices = [j for j in range(len(point)) if j not in nonzero_indices]
                additional_indices = np.random.choice(remaining_indices, 3 - len_nonzero, replace=False)
                current_ternary_space = np.concatenate((nonzero_indices, additional_indices))
        else:
            current_ternary_space = nonzero_indices[:3]

        elements = [dimension_map[j] for j in current_ternary_space]
        used_elements.append(elements)

    return used_elements


# Running the implementation function to test
result = implement_representation_logic(compositions, elementalSpaceComponents)

# Print the result
# for i, elements in enumerate(result):
#     print(f"Composition {i+1}: {elements}")

print(result)

[['Fe', 'Ti', 'V'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['Fe', 'Ni', 'Cr'], ['V', 'Ti', 'Cr'], ['Ni', 'V', 'Cr'], ['Ni', 'V', 'Cr'], ['Ni', 'V', 'Cr'], ['Ni', 'V', 'Cr'], ['Ni'

In [ ]:
t1=equilibrium_callables['equilibrium_callable_NiCrV_0f8b67c1'](compositions[200])
t2=equilibrium_callables['equilibrium_callable_NiCrFe_ce8329fb'](compositions[200])

In [ ]:
print("T1=",t1,"T2=",t2)


T1= {'Phases': ['BCC_A2'], 'PhaseFraction': [1.000000000003023]} T2= {'Phases': ['BCC_A2', 'SIGMA'], 'PhaseFraction': [0.46923805136416286, 0.5307619487955337]}


In [ ]:
while len(queue)>0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [compositions[i] for i in queue]
    if len(queue)>3:
        phases = process_map(equilibrium_callable, elPositions, max_workers=4)
    else:
        phases = [equilibrium_callable(elP) for elP in elPositions]
    feasibilities = [len(set(p) & set(['LAVES_C15', 'LAVES_C36', 'LAVES_C14', 'LIQUID']))==0 and p!=[] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

NameError: name 'elPositions' is not defined

In [ ]:
args = list(zip(elPositions, systems))
phases = process_map(lambda x: combined_callable(*x), args, max_workers=4)

NameError: name 'compositions' is not defined

## Exploring Complex Formed by All 3-Component Alloy Subsystems of 7-Component Space (w. Infeasibility Gliding)

In [48]:
elementalSpaceComponents = ["Ti", "Zr", "Hf", "W", "Nb", "Ta", "Mo"]
attainableSpaceComponents = ["Ti50Zr50", "Hf95Ti5", "NbTaWHf", "Mo80Nb10W10", "TiTa2", "Nb96Mo3W1", "Zr49 Hf1 Mo50"]
attainableSpaceComponentPositions = [
    #Ti, Zr, Hf,  W, Nb, Ta, Mo
    [50, 50,  0,  0,  0,  0,  0], 
    [ 5,  0, 95,  0,  0,  0,  0], 
    [ 0,  0, 25, 25, 25, 25,  0], 
    [ 0,  0,  0, 10, 10,  0, 80],
    [33,  0,  0,  0,  0, 66,  0],
    [ 0,  0,  0,  1, 96,  0,  3],
    [ 0, 49,  1,  0,  0,  0, 50]
]    
ternaries = list(combinations(attainableSpaceComponents, 3))
ternaries_CompPos = list(combinations(attainableSpaceComponentPositions, 3))
ndiv = 8
gridAtt, nList = nimplex.simplex_graph_py(3, ndiv)

for tern, terncp in zip(ternaries, ternaries_CompPos):
    print(f"{str(tern):<40} -> {terncp}")

('Ti50Zr50', 'Hf95Ti5', 'NbTaWHf')       -> ([50, 50, 0, 0, 0, 0, 0], [5, 0, 95, 0, 0, 0, 0], [0, 0, 25, 25, 25, 25, 0])
('Ti50Zr50', 'Hf95Ti5', 'Mo80Nb10W10')   -> ([50, 50, 0, 0, 0, 0, 0], [5, 0, 95, 0, 0, 0, 0], [0, 0, 0, 10, 10, 0, 80])
('Ti50Zr50', 'Hf95Ti5', 'TiTa2')         -> ([50, 50, 0, 0, 0, 0, 0], [5, 0, 95, 0, 0, 0, 0], [33, 0, 0, 0, 0, 66, 0])
('Ti50Zr50', 'Hf95Ti5', 'Nb96Mo3W1')     -> ([50, 50, 0, 0, 0, 0, 0], [5, 0, 95, 0, 0, 0, 0], [0, 0, 0, 1, 96, 0, 3])
('Ti50Zr50', 'Hf95Ti5', 'Zr49 Hf1 Mo50') -> ([50, 50, 0, 0, 0, 0, 0], [5, 0, 95, 0, 0, 0, 0], [0, 49, 1, 0, 0, 0, 50])
('Ti50Zr50', 'NbTaWHf', 'Mo80Nb10W10')   -> ([50, 50, 0, 0, 0, 0, 0], [0, 0, 25, 25, 25, 25, 0], [0, 0, 0, 10, 10, 0, 80])
('Ti50Zr50', 'NbTaWHf', 'TiTa2')         -> ([50, 50, 0, 0, 0, 0, 0], [0, 0, 25, 25, 25, 25, 0], [33, 0, 0, 0, 0, 66, 0])
('Ti50Zr50', 'NbTaWHf', 'Nb96Mo3W1')     -> ([50, 50, 0, 0, 0, 0, 0], [0, 0, 25, 25, 25, 25, 0], [0, 0, 0, 1, 96, 0, 3])
('Ti50Zr50', 'NbTaWHf', 'Zr49 Hf1 Mo5

In [49]:
nList[1]

[0, 2, 9, 10]

In [50]:
# Edges list for graph plotting and path finding purposes
edges = []
# Connectivity list within each subsystem
graphN = [[] for i in range(len(gridAtt * len(ternaries)))]
# Connectivity list between subsystems
graphNS = [[] for i in range(len(graphN))]
compositions = []
# Iterate over ternaries
for i, terncp in enumerate(ternaries_CompPos):
    offset = i*len(gridAtt)
    for j in range(len(gridAtt)):
        for n in nList[j]:
            edges.append((j+offset,n+offset))
            graphN[j+offset].append(n+offset)
    gridAttTemp, gridElTemp = nimplex.embeddedpair_simplex_grid_fractional_py(terncp, ndiv)
    compositions += gridElTemp

In [52]:
type(compositions)

list

In [63]:
find_duplicates(compositions)

'Duplicates found:[0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.0] appears 15 times[0.00625, 0.0, 0.3375, 0.21875, 0.21875, 0.21875, 0.0] appears 5 times[0.0125, 0.0, 0.425, 0.1875, 0.1875, 0.1875, 0.0] appears 5 times[0.018750000000000003, 0.0, 0.5125, 0.15625, 0.15625, 0.15625, 0.0] appears 5 times[0.025, 0.0, 0.6, 0.125, 0.125, 0.125, 0.0] appears 5 times[0.03125, 0.0, 0.6875, 0.09375, 0.09375, 0.09375, 0.0] appears 5 times[0.037500000000000006, 0.0, 0.7749999999999999, 0.0625, 0.0625, 0.0625, 0.0] appears 5 times[0.043750000000000004, 0.0, 0.8624999999999999, 0.03125, 0.03125, 0.03125, 0.0] appears 5 times[0.05, 0.0, 0.95, 0.0, 0.0, 0.0, 0.0] appears 15 times[0.0625, 0.0625, 0.21875, 0.21875, 0.21875, 0.21875, 0.0] appears 5 times[0.10625, 0.0625, 0.8312499999999999, 0.0, 0.0, 0.0, 0.0] appears 5 times[0.125, 0.125, 0.1875, 0.1875, 0.1875, 0.1875, 0.0] appears 5 times[0.1625, 0.125, 0.7124999999999999, 0.0, 0.0, 0.0, 0.0] appears 5 times[0.1875, 0.1875, 0.15625, 0.15625, 0.15625, 0.15625, 0

In [48]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('Hf95Ti5', 'Ti50Zr50'): occurs between ternary [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
('NbTaWHf', 'Ti50Zr50'): occurs between ternary [(0, 5), (0, 6), (0, 7), (0, 8), (5, 6), (5, 7), (5, 8), (6, 7), (6, 8), (7, 8)]
('Hf95Ti5', 'NbTaWHf'): occurs between ternary [(0, 15), (0, 16), (0, 17), (0, 18), (15, 16), (15, 17), (15, 18), (16, 17), (16, 18), (17, 18)]
('Mo80Nb10W10', 'Ti50Zr50'): occurs between ternary [(1, 5), (1, 9), (1, 10), (1, 11), (5, 9), (5, 10), (5, 11), (9, 10), (9, 11), (10, 11)]
('Hf95Ti5', 'Mo80Nb10W10'): occurs between ternary [(1, 15), (1, 19), (1, 20), (1, 21), (15, 19), (15, 20), (15, 21), (19, 20), (19, 21), (20, 21)]
('Ti50Zr50', 'TiTa2'): occurs between ternary [(2, 6), (2, 9), (2, 12), (2, 13), (6, 9), (6, 12), (6, 13), (9, 12), (9, 13), (12, 13)]
('Hf95Ti5', 'TiTa2'): occurs between ternary [(2, 16), (2, 19), (2, 22), (2, 23), (16, 19), (16, 22), (16, 23), (19, 22), (19, 23), (22, 23)]
('Nb96Mo3W1', 'Ti50Zr50'): occur

In [49]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))
            graphNS[i].append(j)

Stitching 0 and 1 at ('Hf95Ti5', 'Ti50Zr50') from [8, 16, 23, 29, 34, 38, 41, 43, 44] to [53, 61, 68, 74, 79, 83, 86, 88, 89]
Stitching 0 and 2 at ('Hf95Ti5', 'Ti50Zr50') from [8, 16, 23, 29, 34, 38, 41, 43, 44] to [98, 106, 113, 119, 124, 128, 131, 133, 134]
Stitching 0 and 3 at ('Hf95Ti5', 'Ti50Zr50') from [8, 16, 23, 29, 34, 38, 41, 43, 44] to [143, 151, 158, 164, 169, 173, 176, 178, 179]
Stitching 0 and 4 at ('Hf95Ti5', 'Ti50Zr50') from [8, 16, 23, 29, 34, 38, 41, 43, 44] to [188, 196, 203, 209, 214, 218, 221, 223, 224]
Stitching 1 and 2 at ('Hf95Ti5', 'Ti50Zr50') from [53, 61, 68, 74, 79, 83, 86, 88, 89] to [98, 106, 113, 119, 124, 128, 131, 133, 134]
Stitching 1 and 3 at ('Hf95Ti5', 'Ti50Zr50') from [53, 61, 68, 74, 79, 83, 86, 88, 89] to [143, 151, 158, 164, 169, 173, 176, 178, 179]
Stitching 1 and 4 at ('Hf95Ti5', 'Ti50Zr50') from [53, 61, 68, 74, 79, 83, 86, 88, 89] to [188, 196, 203, 209, 214, 218, 221, 223, 224]
Stitching 2 and 3 at ('Hf95Ti5', 'Ti50Zr50') from [98, 106, 113

In [50]:
len(edges)

9450

In [54]:
# Plot is quite interesting, being a much more complex version of the last one, but it has too many points for a small Codespace VM to display smootly. You can remove `#` and 
# plotGraph(edges)
clear_output()

In [55]:
from pycalphad import Database
dbf = Database("CrHfMoNbTaTiVWZr_9element_Feb2023.tdb")
phases = list(set(dbf.phases.keys()))
print(elementalSpaceComponents)
print(f'Loaded TDB file with phases considered: {phases}')

['Ti', 'Zr', 'Hf', 'W', 'Nb', 'Ta', 'Mo']
Loaded TDB file with phases considered: ['LAVES_C36', 'HCP_A3', 'LAVES_C15', 'LAVES_C14', 'LIQUID', 'BCC_A2', 'FCC_A1']


In [56]:
from myPycalphadCallable import equilibrium_callable

In [57]:
print(compositions[0])
equilibrium_callable(compositions[0])

[0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.0]


['HCP_A3', 'BCC_A2']

In [58]:
startingNodes = [0, 90, 1527, 1538] + random.sample(range(len(compositions)), 11)
print(f"Starting nodes: {startingNodes}")

for startingNode in startingNodes:
    print(f"Starting node: {compositions[startingNode]}")

Starting nodes: [0, 90, 1527, 1538, 107, 548, 178, 1574, 834, 545, 220, 78, 776, 1098, 1151]
Starting node: [0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.0]
Starting node: [0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.0]
Starting node: [0.0, 0.06125, 0.00125, 0.08750000000000001, 0.08750000000000001, 0.0, 0.7625000000000001]
Starting node: [0.0, 0.0, 0.0, 0.01, 0.96, 0.0, 0.03]
Starting node: [0.375, 0.125, 0.0, 0.0, 0.0, 0.5, 0.0]
Starting node: [0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.0]
Starting node: [0.44375, 0.4375, 0.11875, 0.0, 0.0, 0.0, 0.0]
Starting node: [0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.0]
Starting node: [0.018750000000000003, 0.30625, 0.36249999999999993, 0.0, 0.0, 0.0, 0.3125]
Starting node: [0.20833333333333331, 0.0, 0.0, 0.00375, 0.36, 0.41666666666666663, 0.01125]
Starting node: [0.38125, 0.43625, 0.12, 0.0, 0.0, 0.0, 0.0625]
Starting node: [0.26875, 0.25, 0.35624999999999996, 0.0125, 0.0125, 0.0, 0.1]
Starting

In [59]:
from tqdm.contrib.concurrent import process_map

In [60]:
gridFeasible = [None]*len(compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

In [61]:
while len(queue)>0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [compositions[i] for i in queue]
    if len(queue)>3:
        phases = process_map(equilibrium_callable, elPositions, max_workers=4)
    else:
        phases = [equilibrium_callable(elP) for elP in elPositions]
    feasibilities = [len(set(p) & set(['LAVES_C15', 'LAVES_C36', 'LAVES_C14', 'LIQUID']))==0 and p!=[] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

Queue: [0, 90, 1527, 1538, 107, 548, 178, 1574, 834, 545, 220, 78, 776, 1098, 1151]


  0%|          | 0/15 [00:00<?, ?it/s]

Calculations done: 15    | Explored points: 44   
Queue: [1408, 1, 1537, 1409, 768, 1145, 1152, 775, 1156, 9, 1546, 777, 1157, 271, 783, 784, 277, 406, 280, 1563, 415, 288, 544, 546, 547, 1567, 423, 553, 554, 555, 175, 176, 177, 562, 179, 322, 592, 600, 217, 216, 91, 219, 222, 607, 99, 100, 232, 1003, 108, 1004, 367, 114, 1144, 1273, 1274, 1150, 767]


  0%|          | 0/57 [00:00<?, ?it/s]

Calculations done: 72    | Explored points: 185  
Queue: [1536, 2, 1545, 10, 17, 1553, 1557, 1558, 535, 1562, 1050, 539, 1566, 543, 1570, 552, 560, 561, 568, 580, 583, 591, 599, 92, 606, 1125, 1126, 613, 101, 109, 1135, 1136, 1137, 1138, 625, 115, 1143, 120, 1146, 1149, 1155, 1158, 1160, 1161, 1162, 1165, 1169, 1180, 670, 672, 1187, 1188, 682, 1195, 172, 171, 174, 173, 690, 1210, 212, 213, 727, 729, 730, 735, 231, 1255, 240, 1267, 1270, 1272, 1275, 766, 1278, 1280, 769, 1281, 1282, 772, 1285, 774, 778, 780, 782, 272, 785, 276, 789, 790, 279, 791, 1300, 281, 285, 1312, 289, 294, 295, 1320, 817, 825, 1345, 321, 330, 1365, 1372, 865, 1378, 355, 358, 359, 1390, 366, 1394, 375, 1399, 1401, 1407, 1410, 1415, 1416, 1417, 400, 407, 416, 424, 430, 1455, 997, 1002, 490, 1005, 493, 1010, 1011, 1012]


...

Calculations done: 898   | Explored points: 1222 
Queue: [898, 136, 907, 269, 144, 915, 152, 52, 60, 61, 67, 68, 456, 73, 74, 457, 79, 466, 83, 82, 85, 86, 87, 88, 89, 502]


  0%|          | 0/26 [00:00<?, ?it/s]

Calculations done: 924   | Explored points: 1238 
Queue: [159, 465, 908, 135]


  0%|          | 0/4 [00:00<?, ?it/s]

Calculations done: 928   | Explored points: 1243 


You should see around 930-950 calculations done and around 1250 points explored (thanks to equivalence of stitching points).

In [62]:
feasibleCount = 0
for f in gridFeasible:
    if f:
        feasibleCount += 1
print(f"Feasible points: {feasibleCount} out of {len(gridFeasible)}")

Feasible points: 1039 out of 1575


Out of the explored points, around 1040 should be feasible (i.e., satisfy the equilibrium constraint).

In [63]:
# You can double check the feasibility of the points by running the following code, it is skipped here to save time during the tutorial

#gridPhasesComplete = process_map(equilibrium_callable, compositions)

#gridFeasibleComplete = [len(set(p) & set(['LAVES_C15', 'LAVES_C36', 'LAVES_C14', 'LIQUID']))==0 and p!=[] for p in gridPhasesComplete]
#print(sum(gridFeasibleComplete))

1575

## Adding More Constraints

In [64]:
import pqam_rmsadtandoc2023

def c2rmsad(point):
    formula = ' '.join([f'{c}{p}' for c, p in zip(elementalSpaceComponents, point)])
    return pqam_rmsadtandoc2023.predict(formula)

In [65]:
print(compositions[0])
c2rmsad(compositions[0])

[0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.0]


0.11816658059128823

In [66]:
avgRMSAD = sum([c2rmsad(p) for p in compositions])/len(compositions)
print(f"Average RMSAD: {avgRMSAD}")

Average RMSAD: 0.16141834794438295


In [67]:
startingNodes = [0, 90, 742, 799, 1048, 1070, 1538] + random.sample(range(len(compositions)), 13)

for startingNode in startingNodes:
    print(f"Starting node ({startingNode}): {compositions[startingNode]}")

Starting node (0): [0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.0]
Starting node (90): [0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.0]
Starting node (742): [0.05416666666666667, 0.0, 0.39375, 0.15625, 0.15625, 0.23958333333333331, 0.0]
Starting node (799): [0.025, 0.0, 0.6, 0.125, 0.125, 0.125, 0.0]
Starting node (1048): [0.17291666666666666, 0.18375, 0.1225, 0.0, 0.0, 0.3333333333333333, 0.1875]
Starting node (1070): [0.03125, 0.18375, 0.5975, 0.0, 0.0, 0.0, 0.1875]
Starting node (1538): [0.0, 0.0, 0.0, 0.01, 0.96, 0.0, 0.03]
Starting node (680): [0.0, 0.0, 0.15625, 0.19375, 0.19375, 0.15625, 0.30000000000000004]
Starting node (697): [0.0125, 0.0, 0.39375, 0.16875, 0.16875, 0.15625, 0.1]
Starting node (106): [0.10625, 0.0625, 0.8312499999999999, 0.0, 0.0, 0.0, 0.0]
Starting node (326): [0.0625, 0.0625, 0.0625, 0.06875, 0.6625, 0.0625, 0.01875]
Starting node (276): [0.08333333333333333, 0.0, 0.1875, 0.1875, 0.1875, 0.35416666666666663, 0.0]
Starting node (690): [0.00625, 0.0,

In [68]:
gridFeasible = [None]*len(compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

minRMSAD = 0.2
print(f"Minimum RMSAD: {minRMSAD}")

Minimum RMSAD: 0.2


In [69]:
while len(queue)>0:
    print(f"Queue: {queue}")
    # Skip all points with RMSAD below the minimum (inexpensive) by removing them from the queue, marking them as infeasible, and adding them to explored
    for i in queue:
        if c2rmsad(compositions[i]) < minRMSAD:
            for eq in [i] + graphNS[i]:
                gridFeasible[eq] = False
            explored = explored.union([i] + graphNS[i])
    queue = [i for i in queue if gridFeasible[i] is None]

    # Calculate CALPHAD feasibilities (expensive) of the current queue
    elPositions = [compositions[i] for i in queue]
    if len(queue)>3:
        phases = process_map(equilibrium_callable, elPositions, max_workers=4)
    else:
        phases = [equilibrium_callable(elP) for elP in elPositions]
    feasibilities = [len(set(p) & set(['LAVES_C15', 'LAVES_C36', 'LAVES_C14', 'LIQUID']))==0 and p!=[] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

Queue: [0, 90, 742, 799, 1048, 1070, 1538, 680, 697, 106, 326, 276, 690, 1148, 683, 1437, 502, 335, 3, 893]
Calculations done: 3     | Explored points: 62   
Queue: [97, 98, 803, 195, 838, 105, 843, 142, 113, 150, 793, 187, 798]


  0%|          | 0/12 [00:00<?, ?it/s]

Calculations done: 15    | Explored points: 96   
Queue: [898, 1030, 1032, 141, 149, 792, 157, 797, 802, 805, 806, 940, 944, 1075, 1077, 831, 832, 837, 842, 202, 847, 96, 1120, 104, 112, 119, 760, 763, 764, 895]


  0%|          | 0/24 [00:00<?, ?it/s]

Calculations done: 39    | Explored points: 140  
Queue: [1026, 1027, 1029, 1031, 140, 148, 791, 796, 156, 801, 163, 164, 936, 808, 809, 1071, 1072, 1074, 208, 850, 1116, 1117, 95, 103, 111, 756, 757, 118, 761, 891, 124]


...

Calculations done: 188   | Explored points: 440  
Queue: [391, 392, 393, 395, 398, 656, 657, 661, 289, 290, 297, 303, 308, 438, 443, 341, 342, 348, 605, 611, 483, 616]


  0%|          | 0/12 [00:00<?, ?it/s]

Calculations done: 200   | Explored points: 468  
Queue: [386, 387, 388, 291, 610, 615, 394, 298, 655, 304, 660, 343, 604]


  0%|          | 0/9 [00:00<?, ?it/s]

Calculations done: 209   | Explored points: 482  
Queue: [389, 299, 380, 381, 382]
Calculations done: 209   | Explored points: 488  


In [70]:
sum([f for f in gridFeasible if f])

286

In [78]:
len(gridFeasible)

1575